In [26]:
from langchain.prompts import PipelinePromptTemplate, PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI

reasoning_chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant who provides sentiment analysis."),
    HumanMessagePromptTemplate.from_template("Analyze the following text for sentiment:\nText: {text}")

])

classification_chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant who provides sentiment analysis."),
    HumanMessagePromptTemplate.from_template("Based on the reasoning, classify the sentiment as Positive, Negative, or Neutral.\nReasoning: {reasoning}")
])

llm = ChatOpenAI(model="gpt-4-turbo")

reasoning_chain= LLMChain(llm=llm, prompt=reasoning_chat_template, output_key="reasoning")
classification_chain = LLMChain(llm=llm, prompt=classification_chat_template, output_key="classification")
#final_chain = LLMChain(llm=llm, prompt=final_template, output_key="final_output")

input_text = {"text": "the movie had stunning visual, but the story line was weak"}

sequential_chain = SequentialChain(
    chains=[reasoning_chain, classification_chain],
    input_variables=["text"],
    output_variables=["reasoning", "classification"]
)


pipeline_result = sequential_chain(input_text)

# Access the individual outputs
reasoning_output = pipeline_result["reasoning"]
classification_output = pipeline_result["classification"]

# Display the results
print("Reasoning Output:")
print(reasoning_output)
print("\nClassification Output:")
print(classification_output)

Reasoning Output:
The sentiment of the text is mixed:
- Positive sentiment is expressed towards the "stunning visual."
- Negative sentiment is directed at the "weak" story line.

Classification Output:
Neutral


In [36]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_openai import ChatOpenAI
import json

# Define the ResponseSchemas
sentiment_schema = ResponseSchema(
    name="sentiment",
    description="The sentiment of the text, which can be Positive, Negative, or Neutral."
)

reasoning_schema = ResponseSchema(
    name="reasoning",
    description="The explanation behind why this sentiment was assigned."
)

# Create a StructuredOutputParser
parser = StructuredOutputParser.from_response_schemas([reasoning_schema, sentiment_schema])

# Create format instructions for the parser
format_instructions = parser.get_format_instructions()

# Define the prompt templates
reasoning_chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant who provides sentiment analysis."),
    HumanMessagePromptTemplate.from_template("Analyze the following text for sentiment and provide a detailed reasoning:\n\nText: {text}")
])

classification_chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant who provides sentiment classification."),
    HumanMessagePromptTemplate.from_template(
        "Based on the reasoning provided, classify the sentiment as Positive, Negative, or Neutral. "
        "Use the specified JSON format:\nReasoning: {reasoning}\n{format_instructions}"
    )
])


# Initialize the LLM
llm = ChatOpenAI(model="gpt-4-turbo")

# Create the individual chains
reasoning_chain = LLMChain(llm=llm, prompt=reasoning_chat_template, output_key="reasoning")
classification_chain = LLMChain(llm=llm, prompt=classification_chat_template, output_key="classification")

# Input text for analysis
input_text = {"text": "the movie had stunning visual, but the story line was weak",
              "format_instructions": format_instructions
              }

# Set up the SequentialChain
sequential_chain = SequentialChain(
    chains=[reasoning_chain, classification_chain],
    input_variables=["text", "format_instructions"],
    output_variables=["reasoning", "classification"]
)

# Run the SequentialChain
pipeline_result = sequential_chain(input_text)

# Parse the output using the parser
parsed_output = parser.parse(pipeline_result["classification"])

# Print the final JSON string
print("Final JSON Output:")
print(json.dumps(parsed_output, indent=2))


{'text': 'the movie had stunning visual, but the story line was weak', 'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"reasoning": string  // The explanation behind why this sentiment was assigned.\n\t"sentiment": string  // The sentiment of the text, which can be Positive, Negative, or Neutral.\n}\n```', 'reasoning': 'The sentiment of the text "the movie had stunning visuals, but the storyline was weak" is mixed.\n\n1. **Positive Aspect**: The phrase "stunning visuals" indicates a strong positive sentiment. The word "stunning" is a powerful adjective that conveys exceptional quality or beauty, which suggests that the visual elements of the movie were highly impressive and pleasing.\n\n2. **Negative Aspect**: The contrasting conjunction "but" introduces a negative sentiment, with "the storyline was weak." The word "weak" is a negative adjective that implies 

 MultiPrmoptChain, LLMRouterChain , MULT_PROMPT_ROUTER_TEMPLATE

In [74]:
from langchain.prompts import PromptTemplate
from langchain.chains import MultiPromptChain
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4-turbo")

# Define prompt information for different scenarios
prompt_infos = [
    {
        "name": "Movie Review",
        "description": "Good for analyzing movie reviews",
        "prompt_template": "Analyze the following movie review for sentiment:\n\nReview: {input}"
    },
    {
        "name": "Finance",
        "description": "Good for analyzing financial news",
        "prompt_template": "Analyze the following financial news article for key insights:\n\nArticle: {input}"
    },
    {
        "name": "Science",
        "description": "Good for summarizing scientific articles",
        "prompt_template": "Summarize the following scientific article and highlight the key findings:\n\nArticle: {input}"
    }
]

# Create MultiPromptChain from prompt information
multi_prompt_chain = MultiPromptChain.from_prompts(llm=llm, prompt_infos=prompt_infos)

# Example input text
input_text = {
    "input": "The new movie has stunning visuals but lacks a compelling storyline."
}

# Run the MultiPromptChain
try:
    result = multi_prompt_chain(input_text)
    print("Output:", result)
except Exception as e:
    print(f"Error running MultiPromptChain: {e}")


Output: {'input': 'The new movie has stunning visuals but lacks a compelling storyline.', 'text': 'The sentiment expressed in this movie review is mixed:\n\n1. Positive Aspect: The review starts on a positive note by praising the "stunning visuals" of the movie. This highlights a strong appreciation for the cinematography or visual effects, which is a positive sentiment towards the technical or aesthetic execution of the film.\n\n2. Negative Aspect: The review then points out a significant drawback, stating that the movie "lacks a compelling storyline." This indicates dissatisfaction with the narrative aspect of the movie, suggesting that it fails to engage the viewer fully or effectively in terms of its plot or story development.\n\nOverall, the sentiment of the review balances between appreciation of the visual elements and criticism of the narrative structure, resulting in a mixed overall sentiment.'}
